In [1]:
import os
import re
import pandas as pd
from spacy.lang.en import English
import random

In [2]:
MAIL_DIR = './maildir/'

In [3]:
maildirs = !ls -l $MAIL_DIR
maildirs.pop(0)
maildirs = [line.split(' ')[-1] + "/sent_items/" for line in maildirs]
maildirs[:3]

['allen-p/sent_items/', 'arnold-j/sent_items/', 'arora-h/sent_items/']

In [4]:
unclean_emails = []
for maildir in maildirs:
    if os.path.exists(MAIL_DIR + maildir):
        for doc_name in os.listdir(MAIL_DIR + maildir):
            if os.path.isfile(MAIL_DIR + maildir + doc_name):
                try:
                    with open(MAIL_DIR + maildir + doc_name, "r") as f:
                        unclean_emails.append(f.read())
                except FileNotFoundError:
                    continue
                except UnicodeDecodeError:
                    print(f"Unicode error for {MAIL_DIR + maildir + doc_name}")
    else:
        print(f"SKIPPING: {MAIL_DIR + maildir} -- no such directory")

SKIPPING: ./maildir/dickson-s/sent_items/ -- no such directory
SKIPPING: ./maildir/fossum-d/sent_items/ -- no such directory
SKIPPING: ./maildir/guzman-m/sent_items/ -- no such directory
SKIPPING: ./maildir/hain-m/sent_items/ -- no such directory
SKIPPING: ./maildir/harris-s/sent_items/ -- no such directory
SKIPPING: ./maildir/hyvl-d/sent_items/ -- no such directory
Unicode error for ./maildir/kaminski-v/sent_items/2524.
SKIPPING: ./maildir/linder-e/sent_items/ -- no such directory
SKIPPING: ./maildir/merriss-s/sent_items/ -- no such directory
SKIPPING: ./maildir/rodrique-r/sent_items/ -- no such directory
SKIPPING: ./maildir/south-s/sent_items/ -- no such directory
SKIPPING: ./maildir/stclair-c/sent_items/ -- no such directory
SKIPPING: ./maildir/stokley-c/sent_items/ -- no such directory
SKIPPING: ./maildir/symes-k/sent_items/ -- no such directory
SKIPPING: ./maildir/whalley-l/sent_items/ -- no such directory


In [5]:
def clean_email(text):
    cleaned = []
    lines = text.split("\n")

    header_end_index = -1
    original_message_index = -1
    original_message_found = False
    
    for index, line in enumerate(lines):
        if "X-FileName" in line:
            header_end_index = index
        if "-----" in  line and not original_message_found:
            original_message_index = index
            original_message_found = True

#     return lines[header_end_index+1:len(lines)]
    return "\n".join(lines[header_end_index+1:original_message_index if original_message_index != -1 else len(lines)]).strip()

In [6]:
cleaned_emails = [clean_email(email) for email in unclean_emails]
print(len(cleaned_emails))
cleaned_emails = [email for email in cleaned_emails if len(email.strip()) > 0]
print(len(cleaned_emails))

37919
33533


In [7]:
print(random.choice(cleaned_emails))

Been there, done that.

This doesn't allow us to you have to be based in London!!!


In [8]:
email_df = pd.DataFrame(cleaned_emails, columns=["email_text"])
email_df

,email_text
0,Here is our forecast
1,"Jeff,\n\nIn light of the puny VAR allotments a..."
2,Greg here are my questions regarding the cedar...
3,Is AMF willing to put up the additional earnes...
4,please send me their resumes
...,...
33528,Sundance 3:\tAug. 16 to Sept. 13\nSundance 4:\...
33529,The canadian sap # is \tp00565421\nthe us sap ...
33530,yes
33531,One of us will attend


In [9]:
email_df.to_csv('data/cleaned_emails.csv')